In [15]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()
import collections
import pathlib
import re
import string
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import matplotlib.pyplot as plt
import tensorflow_text as tf_text
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
tfds.disable_progress_bar()


def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [3]:
dataset = preprocessing.text_dataset_from_directory(
    'bbc',
    batch_size=1)

Found 2225 files belonging to 5 classes.


In [24]:
x_tab = []
y_tab = []
for text, label in dataset:
    x_tab.append(np.asarray(text))
    y_tab.append(np.asarray(label))


In [25]:
for index,value in enumerate(y_tab):
    if value==0:
        y_tab[index]=np.array([1,0,0,0,0])
    elif value==1:
        y_tab[index]=np.array([0,1,0,0,0])
    elif value==2:
        y_tab[index]=np.array([0,0,1,0,0])
    elif value==3:
        y_tab[index]=np.array([0,0,0,1,0])
    elif value==4:
        y_tab[index]=np.array([0,0,0,0,1])
x = np.asarray(x_tab)
y = np.asarray(y_tab)

In [26]:
print(np.shape(y_tab))

(2225, 5)


In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=0)

In [28]:
print(type(x_train))
print(type(y_train))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [52]:
VOCAB_SIZE=1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(x_train)

In [53]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'to', 'of', 'and', 'a', 'in', 'for', 'is',
       'that', 'on', 'said', 'it', 'was', 'he', 'be', 'with', 'has', 'as'],
      dtype='<U13')

In [58]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5)
])

In [59]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [60]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [61]:
history = model.fit(x_train, y_train, epochs=10,
                    validation_data=(x_test,y_test), 
                    validation_steps=30)

Epoch 1/10
47/47 [==============================] - 31s 655ms/step - loss: 0.6836 - accuracy: 0.2067 - val_loss: 0.6686 - val_accuracy: 0.2068
Epoch 2/10
47/47 [==============================] - 30s 648ms/step - loss: 0.6099 - accuracy: 0.2396 - val_loss: 0.5123 - val_accuracy: 0.2095
Epoch 3/10
47/47 [==============================] - 29s 614ms/step - loss: 0.5006 - accuracy: 0.2403 - val_loss: 0.4983 - val_accuracy: 0.2490
Epoch 4/10
47/47 [==============================] - 30s 645ms/step - loss: 0.4977 - accuracy: 0.2946 - val_loss: 0.4978 - val_accuracy: 0.2218
Epoch 5/10
47/47 [==============================] - 30s 642ms/step - loss: 0.4971 - accuracy: 0.2403 - val_loss: 0.4976 - val_accuracy: 0.2122
Epoch 6/10
43/47 [==========================>...] - ETA: 2s - loss: 0.4966 - accuracy: 0.2413

KeyboardInterrupt: 